In [1]:
!pip install --upgrade openai gradio pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.94.0
    Uninstalling openai-1.94.0:
      Successfully uninstalled openai-1.94.0
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


LLM-Setup

In [ ]:
import openai

# Your API key
api_key =   # Replace with your OpenAI key

client = openai.OpenAI(api_key=api_key)

# Function to query ChatGPT
def query_chatgpt(prompt, model="gpt-4", temperature=0.7):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=temperature,
            max_tokens=1024
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

prompt = f"""what is can bus """
print(query_chatgpt(prompt))


Dataset-ATT2.json

In [ ]:
import ssl
import certifi
import json
from pymongo import MongoClient

# MongoDB Atlas connection URI
MONGODB_URI = # Replace with your MONGODB URI

# Connect to MongoDB
client = MongoClient(
    MONGODB_URI,
    tls=True,
    tlsCAFile=certifi.where()
)

# Select database and collection
db = client["threat_db"]
prompt_library = db["prompt_library"]

# 1. Predefined prompt templates with aliases
predefined_prompts = [
    {
        "label": "can bus injection",
        "aliases": ["can bus", "controller area network", "can bus attack", "bus injection"],
        "prompt": (
            """Generate an attack tree for the Controller Area Network (CAN) bus in an automotive system. The root node should represent a successful attack on the CAN bus.

The first level of branches should include:
- 'Message Injection',
- 'Denial of Service',
- 'Spoofing',
- 'Eavesdropping',
- 'Fault Injection',
- 'Firmware Manipulation',
- 'Physical Access'.

Use Mermaid format with 'graph TD' syntax.

For each branch, expand with at least two sub-branches on the second level, and for each of those sub-branches, add at least two further sub-branches on the third level, resulting in a three-level hierarchy in total.

Specifically:

- For 'Message Injection', include sub-branches 'Replay Attack' and 'Arbitrary Message Injection'. Then expand 'Replay Attack' with two sub-branches: 'Capturing Packets' and 'Resending Packets'. Expand 'Arbitrary Message Injection' with 'Crafting Messages' and 'Injecting Malicious Commands'.

- For 'Denial of Service', include 'Bus Flooding' and 'Error Frame Injection'. Expand 'Bus Flooding' with 'Continuous Message Sending' and 'Resource Exhaustion'. Expand 'Error Frame Injection' with 'Error Frame Flood' and 'Bus Off State'.

- For 'Spoofing', include 'ID Spoofing' and 'Timing Spoofing'. Expand 'ID Spoofing' with 'Forged IDs' and 'Masquerading'. Expand 'Timing Spoofing' with 'Delay Injection' and 'Replay Timing Manipulation'.

- For 'Eavesdropping', include 'Passive Listening' and 'Data Capture'. Expand 'Passive Listening' with 'Bus Monitoring' and 'Signal Interception'. Expand 'Data Capture' with 'Message Logging' and 'Packet Analysis'.

- For 'Fault Injection', include 'Voltage Manipulation' and 'Clock Glitching'. Expand 'Voltage Manipulation' with 'Power Supply Interruption' and 'Voltage Spike'. Expand 'Clock Glitching' with 'Clock Signal Interference' and 'Timing Violation'.

- For 'Firmware Manipulation', include 'Malicious Firmware Update' and 'Firmware Downgrade'. Expand 'Malicious Firmware Update' with 'Tampered Firmware File' and 'OTA Exploitation'. Expand 'Firmware Downgrade' with 'Rollback Exploit' and 'Signature Bypass'.

- For 'Physical Access', include 'OBD-II Port Exploit' and 'ECU Extraction'. Expand 'OBD-II Port Exploit' with 'Sniffing Traffic' and 'Sending Commands'. Expand 'ECU Extraction' with 'Direct Flash Access' and 'Hardware Debugging Interface'.

Make sure:
- All node labels are short and readable (no long sentences)
- Use unique node IDs or Mermaid-friendly naming (avoid duplicate text nodes)
- No overlapping concepts
- Output is valid Mermaid syntax

Output the full attack tree **only** in valid Mermaid code, wrapped in triple backticks like this:

```mermaid
graph TD
"""

        )
    }
]

# 2. Load additional prompts from a local JSON file (if provided)
def load_prompts_from_file(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            else:
                print("⚠️ JSON file must contain a list of prompts.")
                return []
    except Exception as e:
        print(f"❌ Error reading JSON file: {e}")
        return []

# 3. Merge predefined prompts with JSON prompts
json_prompts = load_prompts_from_file("ATT2.json")  # Update path if needed
all_prompts = predefined_prompts + json_prompts

# 4. Insert or update all prompts
for item in all_prompts:
    if "label" in item and "prompt" in item:
        prompt_library.update_one(
            {"label": item["label"]},
            {"$set": item},
            upsert=True
        )
    else:
        print(f"⚠️ Skipping invalid prompt entry: {item}")

print("✅ Prompt library with aliases populated successfully.")


user interface

In [ ]:
# ========================
# 📦 Library Imports
# ========================
import openai
from openai import OpenAI
import gradio as gr
from pymongo import MongoClient
from datetime import datetime
import re
import csv
import os
import pandas as pd
from collections import defaultdict, deque

# ========================
# 🔐 API & DB Config
# ========================
OPENAI_API_KEY = # Replace with your OpenAI key
MONGODB_URI =# Replace with your MONGODB uri

client_ai = OpenAI(api_key=OPENAI_API_KEY)
mongo_client = MongoClient(MONGODB_URI)
db = mongo_client["threat_db"]
attack_tree_collection = db["attack_trees"]
prompt_library = db["prompt_library"]

EXPORT_DIR = "csv_exports"
os.makedirs(EXPORT_DIR, exist_ok=True)

# ========================
# 🔧 Utility Functions
# ========================

def parse_mermaid_to_named_edges(mermaid_code):
    node_labels = {}
    edges = []
    lines = mermaid_code.splitlines()
    for line in lines:
        node_match = re.findall(r'(\w+)\[(.+?)\]', line)
        for node_id, label in node_match:
            node_labels[node_id.strip()] = label.strip()
    edge_pattern = re.compile(r'(\w+)\s*-->\s*(\w+)')
    for line in lines:
        match = edge_pattern.search(line)
        if match:
            parent_id = match.group(1).strip()
            child_id = match.group(2).strip()
            parent_label = node_labels.get(parent_id, parent_id)
            child_label = node_labels.get(child_id, child_id)
            edges.append((parent_label, child_label))
    return edges

def build_ordered_paths(edges):
    tree = defaultdict(list)
    indegree = defaultdict(int)
    for parent, child in edges:
        tree[parent].append(child)
        indegree[child] += 1
    roots = set(tree.keys()) - set(indegree.keys())
    if not roots:
        return []
    root = list(roots)[0]
    paths = []
    queue = deque([(root, [root])])
    while queue:
        node, path = queue.popleft()
        if node not in tree:
            paths.append(path)
        else:
            for child in tree[node]:
                queue.append((child, path + [child]))
    return paths

def export_structured_csv(label, paths):
    safe_label = label[:30].replace(' ', '_').replace('/', '_')
    filename = f"{safe_label}_{datetime.utcnow().strftime('%Y%m%d_%H%M%S')}.csv"
    filepath = os.path.join(EXPORT_DIR, filename)
    with open(filepath, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(["Surface Goal", "Attack Vector", "Technique", "Method", "Path"])
        for path in paths:
            row = path[:4] + [" > ".join(path)]
            while len(row) < 5:
                row.insert(len(row) - 1, "")
            writer.writerow(row)
    return filepath

def read_csv_as_dataframe(filepath):
    try:
        df = pd.read_csv(filepath)
        df.drop_duplicates(subset=["Path"], inplace=True)
        return df
    except Exception:
        return pd.DataFrame(columns=["Surface Goal", "Attack Vector", "Technique", "Method", "Path"])

# ========================
# 📌 Tab 1: Generate from Label
# ========================
def generate_attack_tree_from_label(label_selected):
    if not label_selected:
        return "❌ Select a threat scenario."
    doc = prompt_library.find_one({"label": label_selected}) or prompt_library.find_one({"aliases": {"$in": [label_selected.lower()]}})
    if not doc or "prompt" not in doc:
        return f"❌ No prompt or alias found for '{label_selected}'"
    matched_prompt = doc["prompt"]
    label_to_save = doc["label"]
    try:
        system_message = {
            "role": "system",
            "content": "You are a cybersecurity expert. Return only the attack tree in Mermaid format using:\n```mermaid\ngraph TD\n...```"
        }
        response = client_ai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[system_message, {"role": "user", "content": matched_prompt}],
            temperature=0.3,
            max_tokens=1500
        )
        raw = response.choices[0].message.content.strip()
        match = re.search(r"```mermaid\s*(graph TD[\s\S]*?)```", raw)
        if not match:
            return "❌ Mermaid diagram not found or invalid format."
        mermaid_code = match.group(1).strip()
        attack_tree_collection.update_one(
            {"label": label_to_save},
            {"$set": {
                "prompt": matched_prompt,
                "mermaid_code": mermaid_code,
                "updated_at": datetime.utcnow()
            }},
            upsert=True
        )
        return f"```mermaid\n{mermaid_code}\n```"
    except Exception as e:
        return f"❌ Error: {str(e)}"

# ========================
# 📜 Tab 2: View Stored Trees
# ========================
def wrapper_load(label):
    if not label:
        return "❌ Select a saved attack tree.", pd.DataFrame(), None
    doc = attack_tree_collection.find_one({"label": label}) or prompt_library.find_one({"aliases": {"$in": [label.lower()]}})
    if doc and "label" in doc and "mermaid_code" not in doc:
        return generate_attack_tree_from_label(doc["label"]), pd.DataFrame(), None
    if not doc or "mermaid_code" not in doc:
        return "❌ No stored attack tree found.", pd.DataFrame(), None
    mermaid_code = doc["mermaid_code"]
    edges = parse_mermaid_to_named_edges(mermaid_code)
    paths = build_ordered_paths(edges)
    csv_path = export_structured_csv(doc["label"], paths)
    df = read_csv_as_dataframe(csv_path)
    return f"```mermaid\n{mermaid_code}\n```", df, csv_path

# ========================
# 📾 Tab 3: Free Prompt
# ========================
def generate_tree_from_free_prompt(prompt):
    if not prompt.strip():
        return "❌ Please enter a valid prompt"
    try:
        all_docs = list(prompt_library.find({}, {"label": 1, "aliases": 1, "prompt": 1, "_id": 0}))
        matched_doc = None
        for doc in all_docs:
            if "label" not in doc:
                continue
            label = doc["label"].lower()
            aliases = [a.lower() for a in doc.get("aliases", [])]
            if label in prompt.lower() or any(alias in prompt.lower() for alias in aliases):
                matched_doc = doc
                break

        if matched_doc:
            base_prompt = matched_doc["prompt"]
            label_to_save = matched_doc["label"]
            extended_prompt = f"{base_prompt}\n\n# Extension:\n{prompt}"
            system_msg = {
                "role": "system",
                "content": "You are a cybersecurity expert. Return the full updated attack tree in Mermaid format:\n```mermaid\ngraph TD\n...```"
            }
            response = client_ai.chat.completions.create(
                model="gpt-4-turbo",
                messages=[system_msg, {"role": "user", "content": extended_prompt}],
                temperature=0.3,
                max_tokens=1200
            )
            raw = response.choices[0].message.content.strip()
            match = re.search(r"```mermaid\s*(graph TD[\s\S]*?)```", raw)
            if not match:
                return "❌ Mermaid diagram not found or invalid format."
            mermaid_code = match.group(1).strip()

            attack_tree_collection.update_one(
                {"label": label_to_save},
                {"$set": {"prompt": extended_prompt, "mermaid_code": mermaid_code, "updated_at": datetime.utcnow()}},
                upsert=True
            )
            prompt_library.update_one(
                {"label": label_to_save},
                {"$set": {"prompt": extended_prompt, "updated_at": datetime.utcnow()}},
                upsert=True
            )
            return f"```mermaid\n{mermaid_code}\n```"

        label_resp = client_ai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "Generate a short, descriptive lowercase label (2–4 words) for this cybersecurity attack surface or scenario."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=12,
            temperature=0.3
        )
        new_label = label_resp.choices[0].message.content.strip().lower().replace(" ", "_")

        alias_resp = client_ai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "Give 3–5 short aliases or alternative names for the following cybersecurity concept or attack surface. Return only a Python list."},
                {"role": "user", "content": new_label.replace("_", " ")}
            ],
            max_tokens=100,
            temperature=0.5
        )
        try:
            alias_text = alias_resp.choices[0].message.content.strip()
            alias_list = eval(alias_text) if alias_text.startswith("[") else [new_label]
            alias_list = [a.lower().strip() for a in alias_list if a.strip()]
            if new_label not in alias_list:
                alias_list.append(new_label)
        except:
            alias_list = [new_label]

        tree_resp = client_ai.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": "Generate a full attack tree for the following prompt in Mermaid format starting with:\n```mermaid\ngraph TD"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1500
        )
        raw_tree = tree_resp.choices[0].message.content.strip()
        match = re.search(r"```mermaid\s*(graph TD[\s\S]*?)```", raw_tree)
        if not match:
            return "❌ Mermaid diagram not found or invalid format."
        mermaid_code = match.group(1).strip()

        prompt_library.insert_one({
            "label": new_label,
            "aliases": alias_list,
            "prompt": prompt,
            "created_at": datetime.utcnow()
        })
        attack_tree_collection.insert_one({
            "label": new_label,
            "prompt": prompt,
            "mermaid_code": mermaid_code,
            "created_at": datetime.utcnow()
        })

        return f"```mermaid\n{mermaid_code}\n```"

    except Exception as e:
        return f"❌ Error: {str(e)}"

# ========================
# 🚀 Dropdown Refresher
# ========================
def get_all_labels():
    return sorted([doc["label"] for doc in prompt_library.find({}, {"label": 1})])

def get_stored_labels():
    return sorted(set([doc["label"] for doc in attack_tree_collection.find({"label": {"$exists": True}})]))

def refresh_dropdowns():
    return gr.update(choices=get_all_labels()), gr.update(choices=get_stored_labels())

# ========================
# 🎨 Gradio UI
# ========================
with gr.Blocks() as demo:
    with gr.Tab("🧠 Generate Attack Tree"):
        gr.Markdown("### 🔐 attack tree")
        label_dropdown = gr.Dropdown(choices=[], label="📌 Select or Type", interactive=True, allow_custom_value=True)
        generate_button = gr.Button("🚀 Generate Attack Tree")
        mermaid_display = gr.Markdown(label="📈 Generated Attack Tree")
        generate_button.click(fn=generate_attack_tree_from_label, inputs=label_dropdown, outputs=mermaid_display)

    with gr.Tab("📂 Library"):
        gr.Markdown("### 📉 View and Export Structured Threat Trees")
        saved_dropdown = gr.Dropdown(choices=[], label="📌 Select Stored Tree", interactive=True, allow_custom_value=True)
        mermaid_output = gr.Markdown(label="📈 Saved Attack Tree")
        relation_table = gr.Dataframe(headers=["Surface Goal", "Attack Vector", "Technique", "Method", "Path"], datatype=["str"]*5, interactive=False)
        download_button = gr.File(label="📅 Download CSV")
        regen_button = gr.Button("🔄 Regenerate Tree from Prompt")
        saved_dropdown.change(fn=wrapper_load, inputs=saved_dropdown, outputs=[mermaid_output, relation_table, download_button])
        regen_button.click(fn=generate_attack_tree_from_label, inputs=saved_dropdown, outputs=mermaid_output)

    with gr.Tab("🗓️ Custom Prompt"):
        gr.Markdown("🔍 Explore and Extend Threat Trees")
        prompt_input = gr.Textbox(label="Enter your Custom Prompt", lines=5, placeholder="e.g. Add another attack vector to CAN Bus")
        custom_mermaid_output = gr.Markdown(label="📌 Extended Attack Tree")
        submit_button = gr.Button("Generate and Update Tree")
        submit_button.click(fn=generate_tree_from_free_prompt, inputs=prompt_input, outputs=custom_mermaid_output).then(
            fn=refresh_dropdowns, inputs=[], outputs=[label_dropdown, saved_dropdown]
        )

    # ⏫ Load choices on app startup
    demo.load(fn=refresh_dropdowns, inputs=[], outputs=[label_dropdown, saved_dropdown])

demo.launch(share=True)
